In [1]:
from fintools.main import Main
from fintools.concurrent import Future
from fintools.rates import Cetes 

In [2]:
Main.hello()

'Hello, world!'

In [3]:
Future

fintools.concurrent.future.Future

In [7]:
import requests
import pandas as pd

In [8]:
data = get_data()
data

,date,value
0,1982-09-02,53.32
5,1982-10-07,40.18
6,1982-10-14,35.07
7,1982-10-21,33.43
9,1982-11-04,37.42
...,...,...
2007,2021-02-18,4.01
2008,2021-02-25,4.02
2009,2021-03-04,4.06
2010,2021-03-11,4.05


In [7]:
from typing import Optional
import datetime as dt

import pandas as pd 
import requests


cetes_tags = {
    "28": "SF43936,CF107,5",
    "91": "SF43939,CF107,9",
    "182": "SF43942,CF107,13"
}

def get_data(s: str) ->  pd.DataFrame:
    url = f"https://www.banxico.org.mx/SieInternet/consultaSerieGrafica.do?s={s}&versionSerie=LA-MAS-RECIENTE&l=es"
    return pd.DataFrame(requests.get(url).json()["valores"], columns=["date", "value"])\
        .query("value > -100")

class Cetes:
    
    def __init__(self, tag: str, date_start: Optional[str] = None, date_end: Optional[str] = None):
        self.series = cetes_tags[tag]
        self.date_start = date_start
        self.date_end = date_end
        self.latest_dataframe = None
        self.latest_request_timestamp = None
        
    def _date_filter(self, data: pd.DataFrame) -> str:
        date_start = data.date.values[0] if self.date_start is None else self.date_start
        date_end = data.date.values[-1] if self.date_end is None else self.date_end
        return "'{start}' <= date <= '{end}'".format(
            start=date_start,
            end=date_end
        )
        
        
    def get_dataframe(self, cache: bool = False):
        if cache and self.latest_dataframe is not None:
            return self.latest_dataframe
        # Get all history
        data = get_data(s=self.series)
        # Get date filter
        date_filter = self._date_filter(data)
        # Update metadata
        self.latest_dataframe =  data.assign(date=pd.to_datetime(data.date, infer_datetime_format=True)).query(date_filter)
        self.latest_request_timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
        return self.latest_dataframe
    
    
class Cetes28(Cetes):
    
    
    def __init__(self, date_start: Optional[str] = None, date_end: Optional[str] = None):
        super().__init__(tag="28", date_start=date_start, date_end=date_end)
        

class Cetes91(Cetes):
    
    def __init__(self, date_start: Optional[str] = None, date_end: Optional[str] = None):
        super().__init__(tag="91", date_start=date_start, date_end=date_end)


class Cetes182(Cetes):
    
    def __init__(self, date_start: Optional[str] = None, date_end: Optional[str] = None):
        super().__init__(tag="182", date_start=date_start, date_end=date_end)

In [6]:

cetes = Cetes28(date_start="2020-12-15")
cetes.get_dataframe()

,date,value
1998,2020-12-17,4.23
1999,2020-12-24,4.27
2000,2020-12-31,4.24
2001,2021-01-07,4.28
2002,2021-01-14,4.19
2003,2021-01-21,4.19
2004,2021-01-28,4.22
2005,2021-02-04,4.24
2006,2021-02-11,4.19
2007,2021-02-18,4.01


In [3]:
cetes.get_dataframe(cache=True)

,date,value
1998,2020-12-17,4.23
1999,2020-12-24,4.27
2000,2020-12-31,4.24
2001,2021-01-07,4.28
2002,2021-01-14,4.19
2003,2021-01-21,4.19
2004,2021-01-28,4.22
2005,2021-02-04,4.24
2006,2021-02-11,4.19
2007,2021-02-18,4.01
